In [10]:
# MILP implementation for dynamic bike rebalancing problem under 4 truck location scenarios

import json
import os
import numpy as np
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpBinary, LpStatus, value
from collections import defaultdict

# === Constants ===
NUM_STATIONS = 30
TIME_SLOT = 60
TOTAL_MINUTES = 180  # e.g., 6:00 to 9:00 => 180 minutes
NUM_PERIODS = TOTAL_MINUTES // TIME_SLOT
TRUCK_CAPACITY = 40
STATION_CAPACITY = {s: 40 if s < 5 else 20 for s in range(NUM_STATIONS)}
NUM_TRUCKS = 3
SCENARIOS = {
    "A": {"city": 3, "outskirts": 0},
    "B": {"city": 2, "outskirts": 1},
    "C": {"city": 1, "outskirts": 2},
    "D": {"city": 0, "outskirts": 3},
}

# === Load and Aggregate Real Demand Data ===
def load_aggregated_demand(data_dir):
    rental_demand = defaultdict(int)
    return_demand = defaultdict(int)
    day_count = 0

    for filename in sorted(os.listdir(data_dir)):
        if filename.startswith("simu0_") and filename.endswith(".json"):
            with open(os.path.join(data_dir, filename), "r") as f:
                trips = json.load(f)
            for td, sd, ta, sa in trips:
                rental_period = int(td) // TIME_SLOT
                return_period = int(ta) // TIME_SLOT
                rental_demand[(int(sd), rental_period)] += 1
                return_demand[(int(sa), return_period)] += 1
            day_count += 1

    f_plus = {(s, t): rental_demand[(s, t)] / day_count for s in range(NUM_STATIONS) for t in range(NUM_PERIODS)}
    f_minus = {(s, t): return_demand[(s, t)] / day_count for s in range(NUM_STATIONS) for t in range(NUM_PERIODS)}
    return f_plus, f_minus

# === MILP Model Creator ===
def create_milp_model(scenario_name, f_plus, f_minus, initial_inventory):
    scenario = SCENARIOS[scenario_name]
    model = LpProblem(f"BSS_Rebalance_{scenario_name}", LpMinimize)

    # === Decision Variables ===
    x_plus = LpVariable.dicts("x_plus", [(s, t) for s in range(NUM_STATIONS) for t in range(NUM_PERIODS)], lowBound=0)
    x_minus = LpVariable.dicts("x_minus", [(s, t) for s in range(NUM_STATIONS) for t in range(NUM_PERIODS)], lowBound=0)
    z = LpVariable.dicts("z", [(s, t, v) for s in range(NUM_STATIONS) for t in range(NUM_PERIODS) for v in range(NUM_TRUCKS)], cat=LpBinary)
    r_plus = LpVariable.dicts("r_plus", [(s, t, v) for s in range(NUM_STATIONS) for t in range(NUM_PERIODS) for v in range(NUM_TRUCKS)], lowBound=0)
    r_minus = LpVariable.dicts("r_minus", [(s, t, v) for s in range(NUM_STATIONS) for t in range(NUM_PERIODS) for v in range(NUM_TRUCKS)], lowBound=0)
    d = LpVariable.dicts("d", [(s, t) for s in range(NUM_STATIONS) for t in range(NUM_PERIODS + 1)], lowBound=0)
    travel = LpVariable.dicts("travel", [(i, j, t, v) for i in range(NUM_STATIONS)
                                                 for j in range(NUM_STATIONS)
                                                 for t in range(NUM_PERIODS - 1)
                                                 for v in range(NUM_TRUCKS)], cat=LpBinary)


    # === Objective Function: minimize unmet demand ===
    model += (
        lpSum((f_plus[s, t] - x_plus[s, t]) + (f_minus[s, t] - x_minus[s, t]) for s in range(NUM_STATIONS) for t in range(NUM_PERIODS))
        + lpSum(dist_matrix[i][j] * travel[i, j, t, v]
                for i in range(NUM_STATIONS)
                for j in range(NUM_STATIONS)
                for t in range(NUM_PERIODS - 1)
                for v in range(NUM_TRUCKS))
    )

    # === Constraint 1: Truck must be at one station per time ===
    for v in range(NUM_TRUCKS):
        for t in range(NUM_PERIODS):
            model += lpSum(z[s, t, v] for s in range(NUM_STATIONS)) == 1

    # === Constraint 2: Initial Truck Location Based on Scenario ===
    assigned = 0
    assigned_stations = set()
    for v in range(NUM_TRUCKS):
        if assigned < scenario["city"]:
            choices = list(set(range(0, 5)) - assigned_stations)
        else:
            choices = list(set(range(5, 30)) - assigned_stations)
        s = choices[0]
        assigned_stations.add(s)
        model += z[s, 0, v] == 1
        for other_s in range(NUM_STATIONS):
            if other_s != s:
                model += z[other_s, 0, v] == 0
        assigned += 1

    # === Constraint 3: Initial Inventory ===
    for s in range(NUM_STATIONS):
        model += d[s, 0] == initial_inventory[s]

    # === Constraint 4: Inventory Flow ===
    for s in range(NUM_STATIONS):
        for t in range(NUM_PERIODS):
            model += d[s, t + 1] == d[s, t] \
                - x_plus[s, t] + x_minus[s, t] \
                - lpSum(r_plus[s, t, v] for v in range(NUM_TRUCKS)) \
                + lpSum(r_minus[s, t, v] for v in range(NUM_TRUCKS))

    # === Constraint 5: Rebalancing limited by truck presence and capacity ===
    for s in range(NUM_STATIONS):
        for t in range(NUM_PERIODS):
            for v in range(NUM_TRUCKS):
                model += r_plus[s, t, v] + r_minus[s, t, v] <= TRUCK_CAPACITY * z[s, t, v]

    # === Constraint 6: Demand limited by available bikes and docks ===
    for s in range(NUM_STATIONS):
        for t in range(NUM_PERIODS):
            model += x_plus[s, t] <= f_plus[s, t]  # cannot serve more than expected rental demand
            model += x_minus[s, t] <= f_minus[s, t]  # cannot serve more than expected return demand
            model += x_plus[s, t] <= d[s, t]  # cannot rent more than available bikes
            model += x_minus[s, t] <= STATION_CAPACITY[s] - d[s, t]  # cannot return if full

    # === Constraint 6: Capacity of stations ===
    for s in range(NUM_STATIONS):
        for t in range(NUM_PERIODS + 1):
            model += d[s, t] <= STATION_CAPACITY[s]
    
    # === Constraint 7: Travel between stations ===
    for v in range(NUM_TRUCKS):
        for t in range(NUM_PERIODS - 1):
            for i in range(NUM_STATIONS):
                model += lpSum(travel[i, j, t, v] for j in range(NUM_STATIONS)) == z[i, t, v]
                model += lpSum(travel[j, i, t, v] for j in range(NUM_STATIONS)) == z[i, t + 1, v]


    return model, x_plus, x_minus


# === Run All Scenarios ===
if __name__ == "__main__":
    data_dir = "./Dataset"
    f_plus, f_minus = load_aggregated_demand(data_dir)
    
    with open(os.path.join(data_dir, "Initial_Inven.json"), "r") as f:
        data = json.load(f)
    initial_inventory = {i: int(val) for i, val in enumerate(data)}
    
    with open(os.path.join(data_dir, "Dis.json"), "r") as f:
        dist_matrix = json.load(f)


    results = {}
    for scenario_name in SCENARIOS:
        model, x_plus, x_minus = create_milp_model(scenario_name, f_plus, f_minus, initial_inventory)
        model.solve()
        results[scenario_name] = {
            "Status": LpStatus[model.status],
            "Total Lost Demand": value(model.objective)
        }

    for k, v in results.items():
        print(f"Scenario {k}: Status = {v['Status']}, Total Lost Demand = {v['Total Lost Demand']:.2f}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Hsuweic/miniconda3/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/j5/b206ytkd1f52lmgd91ffpq1r0000gn/T/2de41fd4fe224e75bfec27e14562ad37-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/j5/b206ytkd1f52lmgd91ffpq1r0000gn/T/2de41fd4fe224e75bfec27e14562ad37-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1334 COLUMNS
At line 29385 RHS
At line 30715 BOUNDS
At line 36386 ENDATA
Problem MODEL has 1329 rows, 6510 columns and 13920 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -53.93 - 0.00 seconds
Cgl0002I 2697 variables fixed
Cgl0004I processed model has 503 rows, 3409 columns (2880 integer (2880 of which binary)) and 7010 elements
Cbc0038I Initial state - 6 integers unsatisfied sum - 0.03
Cbc0038I Pass   1: suminf.    0.00000

In [ ]:
# MILP implementation for dynamic bike rebalancing problem under 4 truck location scenarios

import json
import os
import numpy as np
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpBinary, LpStatus, value
from collections import defaultdict

# === Constants ===
NUM_STATIONS = 30
TIME_SLOT = 60
TOTAL_MINUTES = 1440
NUM_PERIODS = TOTAL_MINUTES // TIME_SLOT
TRUCK_CAPACITY = 40
STATION_CAPACITY = {s: 40 if s < 5 else 20 for s in range(NUM_STATIONS)}
NUM_TRUCKS = 3
SCENARIOS = {
    "A": {"city": 3, "outskirts": 0},
    "B": {"city": 2, "outskirts": 1},
    "C": {"city": 1, "outskirts": 2},
    "D": {"city": 0, "outskirts": 3},
}

# === Load and Aggregate Real Demand Data ===
def load_aggregated_demand(data_dir):
    rental_demand = defaultdict(int)
    return_demand = defaultdict(int)
    day_count = 0

    for filename in sorted(os.listdir(data_dir)):
        if filename.startswith("simu0_") and filename.endswith(".json"):
            with open(os.path.join(data_dir, filename), "r") as f:
                trips = json.load(f)
            for td, sd, ta, sa in trips:
                rental_period = int(td) // TIME_SLOT
                return_period = int(ta) // TIME_SLOT
                rental_demand[(int(sd), rental_period)] += 1
                return_demand[(int(sa), return_period)] += 1
            day_count += 1

    f_plus = {(s, t): rental_demand[(s, t)] / day_count for s in range(NUM_STATIONS) for t in range(NUM_PERIODS)}
    f_minus = {(s, t): return_demand[(s, t)] / day_count for s in range(NUM_STATIONS) for t in range(NUM_PERIODS)}
    return f_plus, f_minus

# === MILP Model Creator ===
def create_milp_model(scenario_name, f_plus, f_minus, initial_inventory, dist_matrix, lambda_dist=0.1):
    scenario = SCENARIOS[scenario_name]
    model = LpProblem(f"BSS_Rebalance_{scenario_name}", LpMinimize)

    # === Variables ===
    x_plus = LpVariable.dicts("x_plus", [(s, t) for s in range(NUM_STATIONS) for t in range(NUM_PERIODS)], lowBound=0)
    x_minus = LpVariable.dicts("x_minus", [(s, t) for s in range(NUM_STATIONS) for t in range(NUM_PERIODS)], lowBound=0)
    r_plus = LpVariable.dicts("r_plus", [(s, t, v) for s in range(NUM_STATIONS) for t in range(NUM_PERIODS) for v in range(NUM_TRUCKS)], lowBound=0)
    r_minus = LpVariable.dicts("r_minus", [(s, t, v) for s in range(NUM_STATIONS) for t in range(NUM_PERIODS) for v in range(NUM_TRUCKS)], lowBound=0)
    d = LpVariable.dicts("d", [(s, t) for s in range(NUM_STATIONS) for t in range(NUM_PERIODS + 1)], lowBound=0)
    d_hat = LpVariable.dicts("d_hat", [(v, t) for v in range(NUM_TRUCKS) for t in range(NUM_PERIODS + 1)], lowBound=0)
    z = LpVariable.dicts("z", [(s, t, v) for s in range(NUM_STATIONS) for t in range(NUM_PERIODS) for v in range(NUM_TRUCKS)], cat=LpBinary)
    travel = LpVariable.dicts("travel", [(i, j, t, v) for i in range(NUM_STATIONS) for j in range(NUM_STATIONS) for t in range(NUM_PERIODS - 1) for v in range(NUM_TRUCKS)], cat=LpBinary)

    # === Objective: lost demand + lambda * total distance ===
    model += (
        lpSum((f_plus[s, t] - x_plus[s, t]) + (f_minus[s, t] - x_minus[s, t])
              for s in range(NUM_STATIONS) for t in range(NUM_PERIODS)) +
        lambda_dist * lpSum(dist_matrix[i][j] * travel[i, j, t, v]
                            for i in range(NUM_STATIONS)
                            for j in range(NUM_STATIONS)
                            for t in range(NUM_PERIODS - 1)
                            for v in range(NUM_TRUCKS))
    )

    # === Constraints ===
    for s in range(NUM_STATIONS):
        model += d[s, 0] == initial_inventory[s]

    for v in range(NUM_TRUCKS):
        model += d_hat[v, 0] == 0

    for v in range(NUM_TRUCKS):
        for t in range(NUM_PERIODS):
            model += lpSum(z[s, t, v] for s in range(NUM_STATIONS)) == 1

    for v in range(NUM_TRUCKS):
        for t in range(NUM_PERIODS - 1):
            for i in range(NUM_STATIONS):
                model += lpSum(travel[i, j, t, v] for j in range(NUM_STATIONS)) == z[i, t, v]
                model += lpSum(travel[j, i, t, v] for j in range(NUM_STATIONS)) == z[i, t + 1, v]

    for s in range(NUM_STATIONS):
        for t in range(NUM_PERIODS):
            model += x_plus[s, t] <= f_plus[s, t]
            model += x_minus[s, t] <= f_minus[s, t]
            model += x_plus[s, t] <= d[s, t]
            model += x_minus[s, t] <= STATION_CAPACITY[s] - d[s, t]
            for v in range(NUM_TRUCKS):
                model += r_plus[s, t, v] + r_minus[s, t, v] <= TRUCK_CAPACITY * z[s, t, v]

    for s in range(NUM_STATIONS):
        for t in range(NUM_PERIODS):
            model += d[s, t + 1] == d[s, t] - x_plus[s, t] + x_minus[s, t] + lpSum(r_minus[s, t, v] - r_plus[s, t, v] for v in range(NUM_TRUCKS))

    for v in range(NUM_TRUCKS):
        for t in range(NUM_PERIODS):
            model += d_hat[v, t + 1] == d_hat[v, t] + lpSum(r_plus[s, t, v] - r_minus[s, t, v] for s in range(NUM_STATIONS))
            model += d_hat[v, t] <= TRUCK_CAPACITY

    for s in range(NUM_STATIONS):
        for t in range(NUM_PERIODS + 1):
            model += d[s, t] <= STATION_CAPACITY[s]

    # === Initial truck location ===
    assigned = 0
    assigned_stations = set()
    for v in range(NUM_TRUCKS):
        if assigned < scenario["city"]:
            choices = list(set(range(0, 5)) - assigned_stations)
        else:
            choices = list(set(range(5, 30)) - assigned_stations)
        s = choices[0]
        assigned_stations.add(s)
        model += z[s, 0, v] == 1
        for other_s in range(NUM_STATIONS):
            if other_s != s:
                model += z[other_s, 0, v] == 0
        assigned += 1

    return model, x_plus, x_minus

# === Run All Scenarios ===
if __name__ == "__main__":
    data_dir = "./Dataset"

    f_plus, f_minus = load_aggregated_demand(data_dir)

    with open(os.path.join(data_dir, "Initial_Inven.json"), "r") as f:
        data = json.load(f)
    initial_inventory = {i: int(val) for i, val in enumerate(data)}

    with open(os.path.join(data_dir, "Dis.json"), "r") as f:
        dist_matrix = json.load(f)

    results = {}
    for scenario_name in SCENARIOS:
        model, x_plus, x_minus = create_milp_model(scenario_name, f_plus, f_minus, initial_inventory, dist_matrix)
        model.solve()
        results[scenario_name] = {
            "Status": LpStatus[model.status],
            "Total Lost Demand": value(model.objective)
        }

    for k, v in results.items():
        print(f"Scenario {k}: Status = {v['Status']}, Total Loss (Demand + Distance) = {v['Total Lost Demand']:.2f}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Hsuweic/miniconda3/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/j5/b206ytkd1f52lmgd91ffpq1r0000gn/T/a2514f6e2664446ca77007093f6e49dd-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/j5/b206ytkd1f52lmgd91ffpq1r0000gn/T/a2514f6e2664446ca77007093f6e49dd-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1355 COLUMNS
At line 29976 RHS
At line 31327 BOUNDS
At line 36998 ENDATA
Problem MODEL has 1350 rows, 6522 columns and 14490 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -53.93 - 0.01 seconds
Cgl0002I 87 variables fixed
Cgl0004I processed model has 500 rows, 3404 columns (2880 integer (2880 of which binary)) and 7359 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -53.93
Cbc0038I

In [2]:
# read json file
def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

data = read_json_file('./Dataset/simu0_0.json')
data



[[4.0, 1.0, 22.0, 11.0],
 [15.0, 1.0, 37.0, 18.0],
 [19.0, 12.0, 45.0, 8.0],
 [48.0, 11.0, 72.0, 5.0],
 [50.0, 22.0, 73.0, 27.0],
 [51.0, 12.0, 75.0, 25.0],
 [54.0, 19.0, 75.0, 29.0],
 [59.0, 13.0, 78.0, 24.0],
 [60.0, 25.0, 65.0, 22.0],
 [68.0, 26.0, 76.0, 7.0],
 [71.0, 5.0, 87.0, 2.0],
 [83.0, 21.0, 108.0, 3.0],
 [87.0, 22.0, 92.0, 16.0],
 [92.0, 14.0, 104.0, 9.0],
 [96.0, 15.0, 114.0, 13.0],
 [97.0, 4.0, 115.0, 10.0],
 [97.0, 8.0, 107.0, 3.0],
 [100.0, 19.0, 113.0, 25.0],
 [105.0, 24.0, 121.0, 17.0],
 [107.0, 21.0, 128.0, 22.0],
 [126.0, 12.0, 155.0, 14.0],
 [155.0, 2.0, 180.0, 29.0],
 [164.0, 22.0, 188.0, 28.0],
 [198.0, 2.0, 208.0, 16.0],
 [198.0, 17.0, 206.0, 21.0],
 [352.0, 12.0, 371.0, 2.0],
 [352.0, 17.0, 366.0, 1.0],
 [360.0, 29.0, 374.0, 24.0],
 [368.0, 28.0, 386.0, 7.0],
 [372.0, 8.0, 383.0, 7.0],
 [372.0, 23.0, 388.0, 7.0],
 [372.0, 28.0, 386.0, 12.0],
 [373.0, 10.0, 379.0, 13.0],
 [373.0, 23.0, 380.0, 3.0],
 [377.0, 9.0, 400.0, 0.0],
 [377.0, 21.0, 388.0, 4.0],
 [378.0, 2